In [1]:
import numpy as np
import matplotlib.pyplot as plt
from data import *
from minibatch import *

%matplotlib inline

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
dataset = get_dataset(type="64")

Total number of images in dataset5:  65774


In [3]:
train_data, train_label_onehot, validation_data, validation_label_onehot, test_data, test_label_onehot \
    = split_train_validation_test(dataset)  

Train data size:  61540
Train label size:  62540
Validation data size:  1000
Validation label size:  1000
Test data size:  3234
Test label size:  3234


In [4]:
X_validation, y_validation = get_data(validation_data, validation_label_onehot)

In [16]:
X_train, y_train = get_data(train_data[:15000], train_label_onehot[:15000])

In [17]:
def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [18]:
def get_mini_batches(data, label, batch_size = 128):
    shuffle_in_unison(data, label)
    n = len(data)
    mini_batches = [(data[k:k+batch_size], label[k:k+batch_size]) for k in range(0, n, batch_size)]
    return mini_batches

mini_batches = get_mini_batches(X_train, y_train)
print(len(mini_batches))

118


#  neural network

In [19]:
from minibatch import *

import tensorflow as tf

print("(WIDTH, HEIGHT)=",(WIDTH,HEIGHT))
C = 24

X = tf.placeholder(tf.float32, [None, WIDTH, HEIGHT, 3])
Y_ = tf.placeholder(tf.float32, [None, C])

# variable learning rate
learning_rate = tf.placeholder(tf.float32)

D = WIDTH*HEIGHT*3 #120*120*3


# Weights initialised with small random values

W1 = tf.Variable(tf.truncated_normal([D, C], stddev=0.01))  # 784 = 28 * 28
#W1 = tf.Variable(tf.zeros((D,C)))
B1 = tf.Variable(tf.zeros([C]))

XX = tf.reshape(X, [-1, D])

Ylogits = tf.matmul(XX, W1) + B1

# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
# TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability
# problems with log(0) which is NaN
softmax = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(softmax)

# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Ylogits, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# training step, the learning rate is a placeholder
train_step = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

saver =tf.train.Saver()

#Initialisation
init = tf.global_variables_initializer()

(WIDTH, HEIGHT)= (64, 64)


In [20]:
import time
BATCH_SIZE = 128
EPOCHS = 100

In [21]:
train_losses = []
train_accuracies = []
validation_accuracies = []

#Initialisation
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    lr = 0.005

    for k in range(EPOCHS):
        
        if lr > 0.0001:
            lr *= 0.99
            
        print(k, ": ", "learning rate = ", lr)
        mini_batches = get_mini_batches(X_train, y_train, batch_size = BATCH_SIZE)
        
        i = 0
        print("--------------------------")
        
        for mini_batch in mini_batches:
            # load batch of images and correct answers

            batch_X, batch_Y = mini_batch
            
            sess.run(train_step, feed_dict={X: batch_X, Y_: batch_Y, learning_rate: lr})

            if i >= 0 and i % 40 == 0:
                train_a, loss = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, Y_: batch_Y})
                train_losses.append(loss)
                train_accuracies.append(train_a)

                valid_a = sess.run(accuracy, feed_dict={X: X_validation, Y_: y_validation})
                validation_accuracies.append(valid_a)

                print(i, ":", " train loss = ", loss, "; train accuracy = ", train_a, "; validation accuracy = ", valid_a)
            
            i+= 1
            
        saver.save(sess, "./tmp/one_layer/model_image64_adam.ckpt")

    # Save the variables to disk.
    save_path = saver.save(sess, "./tmp/one_layer/model_image64_adam.ckpt")
    print("Model saved in file: %s" % save_path)

0 :  learning rate =  0.00495
--------------------------
0 :  train loss =  608.974 ; train accuracy =  0.476562 ; validation accuracy =  0.132
40 :  train loss =  355.33 ; train accuracy =  0.53125 ; validation accuracy =  0.519
80 :  train loss =  378.72 ; train accuracy =  0.625 ; validation accuracy =  0.567
1 :  learning rate =  0.0049005
--------------------------
0 :  train loss =  238.447 ; train accuracy =  0.703125 ; validation accuracy =  0.543
40 :  train loss =  249.679 ; train accuracy =  0.71875 ; validation accuracy =  0.627
80 :  train loss =  144.573 ; train accuracy =  0.734375 ; validation accuracy =  0.612
2 :  learning rate =  0.004851495
--------------------------
0 :  train loss =  183.779 ; train accuracy =  0.742188 ; validation accuracy =  0.566
40 :  train loss =  134.831 ; train accuracy =  0.734375 ; validation accuracy =  0.646
80 :  train loss =  168.64 ; train accuracy =  0.757812 ; validation accuracy =  0.644
3 :  learning rate =  0.00480298005
------

In [ ]:
10000/128

In [ ]:
globals()